In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
from bs4 import BeautifulSoup
import re
import geopy # https://github.com/shakasom/geocoding/blob/master/geocoding.ipynb
from geopy.geocoders import Nominatim
import json
import geopy.distance
import numpy as np
import itertools
import shapely.geometry
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import tabula
import datetime as dt

In [2]:
plt.style.use('default')
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.dpi'] = 200

In [3]:
# Projected Coordinate System (PCS): how you project onto a 2D plane
epsg_2d = 26717 
# Toronto Police Service Open Data includes geographic location information provided in
# the projected coordinate system NAD 1927 UTM 17N
# https://epsg.io/26717

# Geographic Coordinate System (GCS): how coordinates are represented in a 3D space
# World Geodetic System 1984 (WGS84)
# https://epsg.io/4326
epsg_3d = 4326 

# Import TRREB apartment rental transactions

In [4]:
# data before 2012 are in a different format
years = [*range(2012,2022,1)]
quarters = [*range(1,5,1)]

# download public rental market reports from TRREB
# for year in years:
#     for quarter in quarters:
#         response = requests.get(f"https://trreb.ca/files/market-stats/rental-reports/rental_report_Q{quarter}-{year}.pdf")
#         with open(f'./raw_data/rental/Q{quarter}-{year}.pdf', 'wb') as f:
#             f.write(response.content)

In [5]:
dfs = []
for year in years:
    for quarter in quarters:
        # print(year, quarter)
        file1 = f'./raw_data/rental/Q{quarter}-{year}.pdf'
        # focus: apartment
        # table on 3rd page = apartment rental transactions in City of Toronto
        # table on 5th page = townhouse rental transactions in City of Toronto
        table = tabula.read_pdf(file1, pages = '3', stream=True, area = (76.725,31.482,566.775,764.082))[0]
        table.drop(table.index[0:(np.where(table.iloc[:,0] == 'Toronto West')[0][0]-1)], inplace=True)
        table.dropna(axis=1, how='all', inplace=True)
        table.set_index(table.columns[0], inplace=True)
        splits = [table[col].str.split(pat=' ', expand=True).add_prefix(col) for col in table.columns]
        table = pd.concat(splits, axis=1)
        table = table.reset_index(drop=False)
        col1 = ['Region'] + [x for x in ['All_Apartments', 'Bachelor', 'One_Bedroom', 'Two_Bedroom', 'Three_Bedroom'] for _ in (0, 1)]
        col2 = ["", "Total_Listed", "Total_Leased"] + ["Bachelor_Leased", "Avg_Lease_Rate"]*4
        table.columns = ['_'.join([x,y]).strip('_') for x, y in zip(col1,col2)]
        table = table.loc[table['Region'].apply(lambda x: re.search('\d+', x) is not None)]
        table['Region'] = table['Region'].apply(lambda x: re.search('\w\d{2}', x).group())
        table.iloc[:,1:] = table.iloc[:,1:].apply(lambda x_row: pd.Series([int(''.join(re.findall('\d+', x))) if re.findall('\d+', x) != [] else np.nan for x in x_row.values]), axis = 1).values
        table['Year'] = year
        table['Quarter'] = quarter
        table.replace(0, np.nan, inplace=True)
        table.reset_index(drop=True, inplace=True)
        dfs.append(table)

Got stderr: Apr 05, 2022 7:47:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:47:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 05, 2022 7:47:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:47:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 05, 2022 7:47:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:47:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 05, 2022 7:48:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:48:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 05, 2022 7:48:10 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:48:10 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 05, 2022 7:48:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2022 7:48:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

In [6]:
rental = pd.concat(dfs, axis=0)
rental

,Region,All_Apartments_Total_Listed,All_Apartments_Total_Leased,Bachelor_Bachelor_Leased,Bachelor_Avg_Lease_Rate,One_Bedroom_Bachelor_Leased,One_Bedroom_Avg_Lease_Rate,Two_Bedroom_Bachelor_Leased,Two_Bedroom_Avg_Lease_Rate,Three_Bedroom_Bachelor_Leased,Three_Bedroom_Avg_Lease_Rate,Year,Quarter
0,W01,66.0,32.0,NaN,NaN,24.0,1539.0,8.0,2031.0,NaN,NaN,2012,1
1,W02,13.0,9.0,NaN,NaN,3.0,1633.0,6.0,1679.0,NaN,NaN,2012,1
2,W03,3.0,3.0,NaN,NaN,2.0,1050.0,1.0,1650.0,NaN,NaN,2012,1
3,W04,10.0,2.0,NaN,NaN,1.0,1150.0,1.0,1350.0,NaN,NaN,2012,1
4,W05,43.0,17.0,NaN,NaN,12.0,1354.0,5.0,1578.0,NaN,NaN,2012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,E07,125.0,99.0,NaN,NaN,41.0,1976.0,54.0,2408.0,4.0,2645.0,2021,4
31,E08,73.0,35.0,NaN,NaN,12.0,1824.0,16.0,2215.0,7.0,2543.0,2021,4
32,E09,226.0,188.0,5.0,1698.0,111.0,2080.0,68.0,2770.0,4.0,3275.0,2021,4
33,E10,32.0,22.0,NaN,NaN,14.0,1813.0,7.0,2399.0,1.0,2700.0,2021,4


In [7]:
rental.to_csv('./clean_data/rental.csv', index=False)

# Improvements
- Reference for getting data from zumper: https://medium.com/@backupfairuz/exploring-rental-rates-in-canada-hands-on-project-making-use-of-foursquare-api-with-python-413793d2dfca

In [8]:
# Improvement
# bungol.ca for data of housing transaction prices (not listing like other websites)
# real transaction data instead of listings
# access to paid APIs https://rapidapi.com/blog/best-real-estate-apis/